In [ ]:
# STEP 1: Data Cleaning

import pandas as pd
import numpy as np

# 1. Loading the raw CSV file
raw_csv_path = "ṛaw traffic data.csv"
df_raw_traffic = pd.read_csv(r'C:\Users\DELL\Desktop\Capstone-1\ṛaw traffic data.csv')



C:\Users\DELL\AppData\Local\Temp\ipykernel_4416\278613763.py:8: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw_traffic = pd.read_csv(r'C:\Users\DELL\Desktop\Capstone-1\ṛaw traffic data.csv')


In [ ]:
# 2. Creating a new cleaned DataFrame to avoid confusion
df_new_cleaned = df_raw_traffic.copy()

# 3. Droping 'driver_age_raw' column
df_new_cleaned.drop(columns=['driver_age_raw'], inplace=True)

# 4. Converting 'stop_date' to datetime format
df_new_cleaned['stop_date'] = pd.to_datetime(df_new_cleaned['stop_date'], errors='coerce')

# 5. Converting 'stop_time' to Python time object
df_new_cleaned['stop_time'] = pd.to_datetime(df_new_cleaned['stop_time'], format='%H:%M:%S', errors='coerce').dt.time

# 6. Standardize text fields: strip spaces and apply title case
text_columns = [
    'country_name', 'driver_gender', 'driver_race',
    'violation_raw', 'violation', 'search_type',
    'stop_outcome', 'stop_duration'
]

for col in text_columns:
    df_new_cleaned[col] = df_new_cleaned[col].astype(str).str.strip().str.title()

# 7. Replacing empty strings with NaN
df_new_cleaned.replace("", np.nan, inplace=True)

# 8. Droping duplicate rows
df_new_cleaned.drop_duplicates(inplace=True)

# 9. Saving the cleaned DataFrame to a new CSV file
cleaned_output_path = "NewCleaneddata.csv"
df_new_cleaned.to_csv(cleaned_output_path, index=False)

print("✅ Cleaned data saved as NewCleaneddata.csv")


✅ Cleaned data saved as NewCleaneddata.csv


In [3]:
df_new_cleaned.info() #after cleaning

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65538 entries, 0 to 65537
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   stop_date           65538 non-null  datetime64[ns]
 1   stop_time           65538 non-null  object        
 2   country_name        65538 non-null  object        
 3   driver_gender       65538 non-null  object        
 4   driver_age          65538 non-null  int64         
 5   driver_race         65538 non-null  object        
 6   violation_raw       65538 non-null  object        
 7   violation           65538 non-null  object        
 8   search_conducted    65538 non-null  bool          
 9   search_type         65538 non-null  object        
 10  stop_outcome        65538 non-null  object        
 11  is_arrested         65538 non-null  bool          
 12  stop_duration       65538 non-null  object        
 13  drugs_related_stop  65538 non-null  bool      

In [4]:
df_raw_traffic.info() # before cleaning

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65538 entries, 0 to 65537
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   stop_date           65538 non-null  object
 1   stop_time           65538 non-null  object
 2   country_name        65538 non-null  object
 3   driver_gender       65538 non-null  object
 4   driver_age_raw      65538 non-null  int64 
 5   driver_age          65538 non-null  int64 
 6   driver_race         65538 non-null  object
 7   violation_raw       65538 non-null  object
 8   violation           65538 non-null  object
 9   search_conducted    65538 non-null  bool  
 10  search_type         43818 non-null  object
 11  stop_outcome        65538 non-null  object
 12  is_arrested         65538 non-null  bool  
 13  stop_duration       65538 non-null  object
 14  drugs_related_stop  65538 non-null  bool  
 15  vehicle_number      65538 non-null  object
dtypes: bool(3), int64(2), 

In [ ]:
import mysql.connector

# Connecting to MySQL server 
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="09876"  
)

cursor = connection.cursor()

# Creating database if not exists
cursor.execute("CREATE DATABASE IF NOT EXISTS Secureledger")
cursor.execute("USE Secureledger;")


In [ ]:
# Creating the 'policelog' table
create_table_query = """
CREATE TABLE IF NOT EXISTS policelog (
    stop_date DATE,
    stop_time TIME,
    country_name VARCHAR(50),
    driver_gender VARCHAR(10),
    driver_age INT,
    driver_race VARCHAR(50),
    violation_raw VARCHAR(100),
    violation VARCHAR(100),
    search_conducted BOOLEAN,
    search_type VARCHAR(100),
    stop_outcome VARCHAR(50),
    is_arrested BOOLEAN,
    stop_duration VARCHAR(20),
    drugs_related_stop BOOLEAN,
    vehicle_number VARCHAR(50)
)
"""
cursor.execute(create_table_query)
connection.commit()


In [ ]:
import pandas as pd

# Loading the cleaned CSV file
data_to_insert = pd.read_csv("NewCleaneddata.csv", parse_dates=['stop_date'])

# Converting stop_time string to time object
data_to_insert['stop_time'] = pd.to_datetime(data_to_insert['stop_time'], format='%H:%M:%S', errors='coerce').dt.time

# Preparing SQL insert query
insert_query = """
INSERT INTO policelog (
    stop_date, stop_time, country_name, driver_gender, driver_age, driver_race,
    violation_raw, violation, search_conducted, search_type, stop_outcome,
    is_arrested, stop_duration, drugs_related_stop, vehicle_number
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Inserting each row
for _, row in data_to_insert.iterrows():
    values = tuple(row)
    cursor.execute(insert_query, values)

connection.commit()
cursor.close()
connection.close()

print("✅ Data successfully inserted into Secureledger.policelog")


✅ Data successfully inserted into Secureledger.policelog


In [ ]:
#SQL Queries#13/7/25

In [10]:
!pip install tabulate

  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)


In [ ]:
import mysql.connector
from tabulate import tabulate

# Connecting to MySQL once
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="09876",
    database="Secureledger"
)
cursor = conn.cursor()

In [ ]:
# ✅ Query 1 – Vehicle-Based
# What are the top 10 vehicle_Number involved in drug-related stops?
cursor.execute("""
SELECT vehicle_number, COUNT(*) AS drug_stop_count
FROM policelog
WHERE drugs_related_stop = TRUE
GROUP BY vehicle_number
ORDER BY drug_stop_count DESC
LIMIT 10;
""")
results = cursor.fetchall()
headers = ["Vehicle Number", "Drug Stop Count"]
print(tabulate(results, headers=headers, tablefmt="grid"))


+------------------+-------------------+
| Vehicle Number   |   Drug Stop Count |
+==================+===================+
| UP76DY3473       |                 1 |
+------------------+-------------------+
| RJ83PZ4441       |                 1 |
+------------------+-------------------+
| RJ32OM7264       |                 1 |
+------------------+-------------------+
| RJ76TI3807       |                 1 |
+------------------+-------------------+
| DL75KZ7835       |                 1 |
+------------------+-------------------+
| DL50PO5101       |                 1 |
+------------------+-------------------+
| UP67CQ9426       |                 1 |
+------------------+-------------------+
| KA61JB1004       |                 1 |
+------------------+-------------------+
| WB70IV9884       |                 1 |
+------------------+-------------------+
| WB75TF1052       |                 1 |
+------------------+-------------------+


In [ ]:
# ✅ Query 2 – Vehicle-Based
#  Which vehicles were most frequently searched?
cursor.execute("""
SELECT vehicle_number, COUNT(*) AS search_count
FROM policelog
WHERE search_conducted = TRUE
GROUP BY vehicle_number
ORDER BY search_count DESC
LIMIT 10;
""")
results = cursor.fetchall()
headers = ["Vehicle Number", "Search Count"]
print(tabulate(results, headers=headers, tablefmt="grid"))


+------------------+----------------+
| Vehicle Number   |   Search Count |
+==================+================+
| UP76DY3473       |              1 |
+------------------+----------------+
| RJ76TI3807       |              1 |
+------------------+----------------+
| MH89ZF7676       |              1 |
+------------------+----------------+
| DL50PO5101       |              1 |
+------------------+----------------+
| KA79RD2271       |              1 |
+------------------+----------------+
| WB75TF1052       |              1 |
+------------------+----------------+
| UP82DB7554       |              1 |
+------------------+----------------+
| MH15GS2782       |              1 |
+------------------+----------------+
| MH80WR1351       |              1 |
+------------------+----------------+
| WB35BJ2947       |              1 |
+------------------+----------------+


In [ ]:
# ✅ Query 3 – Demographic-Based
#  Which driver age group had the highest arrest rate?
cursor.execute("""
SELECT driver_age, 
       COUNT(*) AS total_stops,
       SUM(is_arrested) AS total_arrests,
       ROUND(SUM(is_arrested)/COUNT(*) * 100, 2) AS arrest_rate
FROM policelog
GROUP BY driver_age
HAVING COUNT(*) > 5
ORDER BY arrest_rate DESC
LIMIT 10;
""")
results = cursor.fetchall()
headers = ["Driver Age", "Total Stops", "Total Arrests", "Arrest Rate (%)"]
print(tabulate(results, headers=headers, tablefmt="grid"))


+--------------+---------------+-----------------+-------------------+
|   Driver Age |   Total Stops |   Total Arrests |   Arrest Rate (%) |
+==============+===============+=================+===================+
|           53 |          1025 |             548 |             53.46 |
+--------------+---------------+-----------------+-------------------+
|           26 |          1040 |             555 |             53.37 |
+--------------+---------------+-----------------+-------------------+
|           67 |           998 |             532 |             53.31 |
+--------------+---------------+-----------------+-------------------+
|           28 |          1100 |             580 |             52.73 |
+--------------+---------------+-----------------+-------------------+
|           18 |          1008 |             529 |             52.48 |
+--------------+---------------+-----------------+-------------------+
|           21 |          1051 |             549 |             52.24 |
+-----

In [ ]:
# ✅ Query 4 – Demographic-Based
# What is the gender distribution of drivers stopped in each country?
cursor.execute("""
SELECT country_name, driver_gender, COUNT(*) AS count
FROM policelog
GROUP BY country_name, driver_gender
ORDER BY country_name;
""")
results = cursor.fetchall()
headers = ["Country", "Gender", "Stop Count"]
print(tabulate(results, headers=headers, tablefmt="grid"))


+-----------+----------+--------------+
| Country   | Gender   |   Stop Count |
+===========+==========+==============+
| Canada    | F        |        10997 |
+-----------+----------+--------------+
| Canada    | M        |        10911 |
+-----------+----------+--------------+
| India     | F        |        11043 |
+-----------+----------+--------------+
| India     | M        |        10955 |
+-----------+----------+--------------+
| Usa       | F        |        10841 |
+-----------+----------+--------------+
| Usa       | M        |        10791 |
+-----------+----------+--------------+


In [ ]:
# ✅ Query 5 – Demographic-Based
#  Which race and gender combination has the highest search rate?
cursor.execute("""
SELECT driver_race, driver_gender, 
       COUNT(*) AS total_stops,
       SUM(search_conducted = TRUE) AS total_searches,
       ROUND(SUM(search_conducted = TRUE)/COUNT(*) * 100, 2) AS search_rate
FROM policelog
GROUP BY driver_race, driver_gender
HAVING COUNT(*) > 5
ORDER BY search_rate DESC
LIMIT 10;
""")
results = cursor.fetchall()
headers = ["Race", "Gender", "Total Stops", "Searches", "Search Rate (%)"]
print(tabulate(results, headers=headers, tablefmt="grid"))


+----------+----------+---------------+------------+-------------------+
| Race     | Gender   |   Total Stops |   Searches |   Search Rate (%) |
+==========+==========+===============+============+===================+
| Asian    | F        |          6649 |       3410 |             51.29 |
+----------+----------+---------------+------------+-------------------+
| Hispanic | F        |          6567 |       3326 |             50.65 |
+----------+----------+---------------+------------+-------------------+
| Other    | F        |          6524 |       3294 |             50.49 |
+----------+----------+---------------+------------+-------------------+
| Asian    | M        |          6462 |       3239 |             50.12 |
+----------+----------+---------------+------------+-------------------+
| Hispanic | M        |          6439 |       3212 |             49.88 |
+----------+----------+---------------+------------+-------------------+
| White    | M        |          6546 |       3262 

In [ ]:
# ✅ Query 6 – Time-Based
#  What time of day sees the most traffic stops?
cursor.execute("""
SELECT HOUR(stop_time) AS hour_of_day, COUNT(*) AS stop_count
FROM policelog
GROUP BY hour_of_day
ORDER BY stop_count DESC;
""")
results = cursor.fetchall()
headers = ["Hour", "Stop Count"]
print(tabulate(results, headers=headers, tablefmt="grid"))


+--------+--------------+
|   Hour |   Stop Count |
+========+==============+
|      0 |         2760 |
+--------+--------------+
|      1 |         2760 |
+--------+--------------+
|      2 |         2760 |
+--------+--------------+
|      3 |         2760 |
+--------+--------------+
|      4 |         2760 |
+--------+--------------+
|      5 |         2760 |
+--------+--------------+
|      6 |         2760 |
+--------+--------------+
|      7 |         2760 |
+--------+--------------+
|      8 |         2760 |
+--------+--------------+
|      9 |         2760 |
+--------+--------------+
|     10 |         2760 |
+--------+--------------+
|     11 |         2760 |
+--------+--------------+
|     12 |         2718 |
+--------+--------------+
|     13 |         2700 |
+--------+--------------+
|     14 |         2700 |
+--------+--------------+
|     15 |         2700 |
+--------+--------------+
|     16 |         2700 |
+--------+--------------+
|     17 |         2700 |
+--------+--

In [ ]:
# ✅ Query 7 – Time-Based
# What is the average stop duration for different violations?
cursor.execute("""
SELECT violation, AVG(CASE
    WHEN stop_duration = '0-15 Min' THEN 7
    WHEN stop_duration = '16-30 Min' THEN 23
    WHEN stop_duration = '30+ Min' THEN 45
    ELSE 0 END) AS avg_duration_est
FROM policelog
GROUP BY violation
ORDER BY avg_duration_est DESC;
""")
results = cursor.fetchall()
headers = ["Violation", "Avg Duration (min)"]
print(tabulate(results, headers=headers, tablefmt="grid"))


+-------------+----------------------+
| Violation   |   Avg Duration (min) |
+=============+======================+
| Other       |              25.5531 |
+-------------+----------------------+
| Seatbelt    |              25.0068 |
+-------------+----------------------+
| Speeding    |              24.9209 |
+-------------+----------------------+
| Dui         |              24.8842 |
+-------------+----------------------+
| Signal      |              24.8377 |
+-------------+----------------------+


In [ ]:
# ✅ Query 8 – Time-Based
#  Are stops during the night more likely to lead to arrests?
cursor.execute("""
SELECT 
  CASE 
    WHEN HOUR(stop_time) BETWEEN 0 AND 5 THEN 'Night'
    WHEN HOUR(stop_time) BETWEEN 6 AND 17 THEN 'Day'
    ELSE 'Evening'
  END AS time_period,
  COUNT(*) AS total_stops,
  SUM(is_arrested) AS total_arrests,
  ROUND(SUM(is_arrested)/COUNT(*) * 100, 2) AS arrest_rate
FROM policelog
GROUP BY time_period;
""")
results = cursor.fetchall()
headers = ["Time Period", "Total Stops", "Total Arrests", "Arrest Rate (%)"]
print(tabulate(results, headers=headers, tablefmt="grid"))


+---------------+---------------+-----------------+-------------------+
| Time Period   |   Total Stops |   Total Arrests |   Arrest Rate (%) |
+===============+===============+=================+===================+
| Night         |         16560 |            8151 |             49.22 |
+---------------+---------------+-----------------+-------------------+
| Day           |         32778 |           16557 |             50.51 |
+---------------+---------------+-----------------+-------------------+
| Evening       |         16200 |            8138 |             50.23 |
+---------------+---------------+-----------------+-------------------+


In [ ]:
# ✅ Query 9 – Violation-Based
#  Which violations are most associated with searches or arrests?
cursor.execute("""
SELECT violation,
       COUNT(*) AS total_stops,
       SUM(search_conducted) AS total_searches,
       SUM(is_arrested) AS total_arrests
FROM policelog
GROUP BY violation
ORDER BY total_arrests DESC
LIMIT 10;
""")
results = cursor.fetchall()
headers = ["Violation", "Total Stops", "Searches", "Arrests"]
print(tabulate(results, headers=headers, tablefmt="grid"))


+-------------+---------------+------------+-----------+
| Violation   |   Total Stops |   Searches |   Arrests |
+=============+===============+============+===========+
| Dui         |         13075 |       6481 |      6610 |
+-------------+---------------+------------+-----------+
| Speeding    |         13150 |       6573 |      6598 |
+-------------+---------------+------------+-----------+
| Seatbelt    |         13007 |       6562 |      6573 |
+-------------+---------------+------------+-----------+
| Signal      |         13112 |       6509 |      6549 |
+-------------+---------------+------------+-----------+
| Other       |         13194 |       6576 |      6516 |
+-------------+---------------+------------+-----------+


In [ ]:
# ✅ Query 10 – Violation-Based
#  Which violations are most common among younger drivers (<25)?
cursor.execute("""
SELECT violation, COUNT(*) AS count
FROM policelog
WHERE driver_age < 25
GROUP BY violation
ORDER BY count DESC
LIMIT 10;
""")
results = cursor.fetchall()
headers = ["Violation", "Count"]
print(tabulate(results, headers=headers, tablefmt="grid"))


+-------------+---------+
| Violation   |   Count |
+=============+=========+
| Speeding    |    1476 |
+-------------+---------+
| Signal      |    1427 |
+-------------+---------+
| Other       |    1422 |
+-------------+---------+
| Seatbelt    |    1420 |
+-------------+---------+
| Dui         |    1359 |
+-------------+---------+


In [ ]:
# ✅ Query 11 – Violation-Based
#  Is there a violation that rarely results in search or arrest?
cursor.execute("""
SELECT violation,
       COUNT(*) AS total_stops,
       SUM(search_conducted) AS total_searches,
       SUM(is_arrested) AS total_arrests
FROM policelog
GROUP BY violation
HAVING total_searches = 0 AND total_arrests = 0;
""")
results = cursor.fetchall()
headers = ["Violation", "Total Stops", "Searches", "Arrests"]
print(tabulate(results, headers=headers, tablefmt="grid"))


+-------------+---------------+------------+-----------+
| Violation   | Total Stops   | Searches   | Arrests   |
+=============+===============+============+===========+
+-------------+---------------+------------+-----------+


In [ ]:
# ✅ Query 12 – Location-Based
#  Which countries report the highest rate of drug-related stops?
cursor.execute("""
SELECT country_name,
       COUNT(*) AS total_stops,
       SUM(drugs_related_stop = TRUE) AS drug_stops,
       ROUND(SUM(drugs_related_stop = TRUE)/COUNT(*) * 100, 2) AS drug_stop_rate
FROM policelog
GROUP BY country_name
ORDER BY drug_stop_rate DESC
LIMIT 10;
""")
results = cursor.fetchall()
headers = ["Country", "Total Stops", "Drug Stops", "Drug Stop Rate (%)"]
print(tabulate(results, headers=headers, tablefmt="grid"))


+-----------+---------------+--------------+----------------------+
| Country   |   Total Stops |   Drug Stops |   Drug Stop Rate (%) |
+===========+===============+==============+======================+
| Usa       |         21632 |        10896 |                50.37 |
+-----------+---------------+--------------+----------------------+
| Canada    |         21908 |        10976 |                50.1  |
+-----------+---------------+--------------+----------------------+
| India     |         21998 |        10897 |                49.54 |
+-----------+---------------+--------------+----------------------+


In [ ]:
# ✅ Query 13 – Location-Based
#  What is the arrest rate by country and violation?
cursor.execute("""
SELECT country_name, violation,
       COUNT(*) AS total_stops,
       SUM(is_arrested) AS total_arrests,
       ROUND(SUM(is_arrested)/COUNT(*) * 100, 2) AS arrest_rate
FROM policelog
GROUP BY country_name, violation
ORDER BY arrest_rate DESC
LIMIT 15;
""")
results = cursor.fetchall()
headers = ["Country", "Violation", "Total Stops", "Total Arrests", "Arrest Rate (%)"]
print(tabulate(results, headers=headers, tablefmt="grid"))


+-----------+-------------+---------------+-----------------+-------------------+
| Country   | Violation   |   Total Stops |   Total Arrests |   Arrest Rate (%) |
+===========+=============+===============+=================+===================+
| Canada    | Dui         |          4414 |            2257 |             51.13 |
+-----------+-------------+---------------+-----------------+-------------------+
| India     | Dui         |          4420 |            2247 |             50.84 |
+-----------+-------------+---------------+-----------------+-------------------+
| India     | Speeding    |          4358 |            2212 |             50.76 |
+-----------+-------------+---------------+-----------------+-------------------+
| India     | Signal      |          4444 |            2251 |             50.65 |
+-----------+-------------+---------------+-----------------+-------------------+
| Usa       | Seatbelt    |          4241 |            2145 |             50.58 |
+-----------+---

In [ ]:
# ✅ Query 14 – Location-Based
#  Which country has the most stops with search conducted?
cursor.execute("""
SELECT country_name, COUNT(*) AS search_count
FROM policelog
WHERE search_conducted = TRUE
GROUP BY country_name
ORDER BY search_count DESC
LIMIT 10;
""")
results = cursor.fetchall()
headers = ["Country", "Search Count"]
print(tabulate(results, headers=headers, tablefmt="grid"))


+-----------+----------------+
| Country   |   Search Count |
+===========+================+
| Canada    |          11020 |
+-----------+----------------+
| India     |          10959 |
+-----------+----------------+
| Usa       |          10722 |
+-----------+----------------+


In [ ]:
# ✅ Query 15 – Complex
#  Yearly Breakdown of Stops and Arrests by Country (Using Subquery and Window Functions)
cursor.execute("""
WITH yearly_counts AS (
    SELECT country_name, YEAR(stop_date) AS year,
           COUNT(*) AS total_stops,
           SUM(is_arrested) AS total_arrests
    FROM policelog
    GROUP BY country_name, year
)
SELECT country_name, year, total_stops, total_arrests,
       ROUND((total_arrests / total_stops) * 100, 2) AS arrest_rate,
       RANK() OVER (PARTITION BY country_name ORDER BY year) AS year_rank
FROM yearly_counts
ORDER BY country_name, year;
""")
results = cursor.fetchall()
headers = ["Country", "Year", "Total Stops", "Total Arrests", "Arrest Rate (%)", "Year Rank"]
print(tabulate(results, headers=headers, tablefmt="grid"))


+-----------+--------+---------------+-----------------+-------------------+-------------+
| Country   |   Year |   Total Stops |   Total Arrests |   Arrest Rate (%) |   Year Rank |
+===========+========+===============+=================+===================+=============+
| Canada    |   2020 |         21908 |           10999 |             50.21 |           1 |
+-----------+--------+---------------+-----------------+-------------------+-------------+
| India     |   2020 |         21998 |           11091 |             50.42 |           1 |
+-----------+--------+---------------+-----------------+-------------------+-------------+
| Usa       |   2020 |         21632 |           10756 |             49.72 |           1 |
+-----------+--------+---------------+-----------------+-------------------+-------------+


In [ ]:
# ✅ Query 16 – Complex
#  Driver Violation Trends Based on Age and Race (Join with Subquery)
cursor.execute("""
WITH age_race_summary AS (
    SELECT DISTINCT driver_age, driver_race
    FROM policelog
    WHERE driver_age IS NOT NULL AND driver_race IS NOT NULL
)
SELECT a.driver_age, a.driver_race, p.violation, COUNT(*) AS violation_count
FROM age_race_summary a
JOIN policelog p
  ON a.driver_age = p.driver_age AND a.driver_race = p.driver_race
GROUP BY a.driver_age, a.driver_race, p.violation
ORDER BY violation_count DESC
LIMIT 15;
""")
results = cursor.fetchall()
headers = ["Driver Age", "Race", "Violation", "Count"]
print(tabulate(results, headers=headers, tablefmt="grid"))


+--------------+--------+-------------+---------+
|   Driver Age | Race   | Violation   |   Count |
+==============+========+=============+=========+
|           56 | White  | Dui         |      63 |
+--------------+--------+-------------+---------+
|           71 | Other  | Speeding    |      59 |
+--------------+--------+-------------+---------+
|           61 | White  | Dui         |      59 |
+--------------+--------+-------------+---------+
|           78 | Asian  | Dui         |      59 |
+--------------+--------+-------------+---------+
|           39 | White  | Other       |      58 |
+--------------+--------+-------------+---------+
|           54 | Black  | Other       |      58 |
+--------------+--------+-------------+---------+
|           33 | Other  | Speeding    |      58 |
+--------------+--------+-------------+---------+
|           70 | Black  | Dui         |      58 |
+--------------+--------+-------------+---------+
|           79 | Asian  | Other       |      58 |


In [ ]:
# ✅ Query 17 – Complex
#  Time Period Analysis of Stops (Joining with Date Functions)
cursor.execute("""
WITH time_details AS (
    SELECT stop_date, stop_time,
           YEAR(stop_date) AS yr,
           MONTH(stop_date) AS mnth,
           HOUR(stop_time) AS hr
    FROM policelog
),
date_summary AS (
    SELECT yr, mnth, hr, COUNT(*) AS stop_count
    FROM time_details
    GROUP BY yr, mnth, hr
)
SELECT t.yr, t.mnth, t.hr, t.stop_count
FROM date_summary t
JOIN time_details td
  ON YEAR(td.stop_date) = t.yr AND MONTH(td.stop_date) = t.mnth AND HOUR(td.stop_time) = t.hr
GROUP BY t.yr, t.mnth, t.hr, t.stop_count
ORDER BY t.yr, t.mnth, t.hr;
""")
results = cursor.fetchall()
headers = ["Year", "Month", "Hour", "Stop Count"]
print(tabulate(results, headers=headers, tablefmt="grid"))


+--------+---------+--------+--------------+
|   Year |   Month |   Hour |   Stop Count |
+========+=========+========+==============+
|   2020 |       1 |      0 |         1860 |
+--------+---------+--------+--------------+
|   2020 |       1 |      1 |         1860 |
+--------+---------+--------+--------------+
|   2020 |       1 |      2 |         1860 |
+--------+---------+--------+--------------+
|   2020 |       1 |      3 |         1860 |
+--------+---------+--------+--------------+
|   2020 |       1 |      4 |         1860 |
+--------+---------+--------+--------------+
|   2020 |       1 |      5 |         1860 |
+--------+---------+--------+--------------+
|   2020 |       1 |      6 |         1860 |
+--------+---------+--------+--------------+
|   2020 |       1 |      7 |         1860 |
+--------+---------+--------+--------------+
|   2020 |       1 |      8 |         1860 |
+--------+---------+--------+--------------+
|   2020 |       1 |      9 |         1860 |
+--------+

In [ ]:
# ✅ Query 18 – Complex
#  Violations with High Search and Arrest Rates (Window Function)
cursor.execute("""
WITH violation_rates AS (
    SELECT violation,
           COUNT(*) AS total_stops,
           SUM(search_conducted) AS total_searches,
           SUM(is_arrested) AS total_arrests,
           ROUND(SUM(search_conducted)/COUNT(*) * 100, 2) AS search_rate,
           ROUND(SUM(is_arrested)/COUNT(*) * 100, 2) AS arrest_rate
    FROM policelog
    GROUP BY violation
    HAVING COUNT(*) > 10
),
ranked_violations AS (
    SELECT *,
           RANK() OVER (ORDER BY arrest_rate DESC) AS arrest_rank
    FROM violation_rates
)
SELECT violation, total_stops, total_searches, total_arrests, search_rate, arrest_rate
FROM ranked_violations
ORDER BY arrest_rank
LIMIT 10;
""")
results = cursor.fetchall()
headers = ["Violation", "Total Stops", "Searches", "Arrests", "Search Rate (%)", "Arrest Rate (%)"]
print(tabulate(results, headers=headers, tablefmt="grid"))


+-------------+---------------+------------+-----------+-------------------+-------------------+
| Violation   |   Total Stops |   Searches |   Arrests |   Search Rate (%) |   Arrest Rate (%) |
+=============+===============+============+===========+===================+===================+
| Dui         |         13075 |       6481 |      6610 |             49.57 |             50.55 |
+-------------+---------------+------------+-----------+-------------------+-------------------+
| Seatbelt    |         13007 |       6562 |      6573 |             50.45 |             50.53 |
+-------------+---------------+------------+-----------+-------------------+-------------------+
| Speeding    |         13150 |       6573 |      6598 |             49.98 |             50.17 |
+-------------+---------------+------------+-----------+-------------------+-------------------+
| Signal      |         13112 |       6509 |      6549 |             49.64 |             49.95 |
+-------------+---------------

In [ ]:
# ✅ Query 19 – Complex
#  Driver Demographics by Country (Age, Gender, and Race)
cursor.execute("""
SELECT country_name, driver_age, driver_gender, driver_race, COUNT(*) AS count
FROM policelog
WHERE driver_age IS NOT NULL AND driver_gender IS NOT NULL AND driver_race IS NOT NULL
GROUP BY country_name, driver_age, driver_gender, driver_race
ORDER BY count DESC
LIMIT 15;
""")
results = cursor.fetchall()
headers = ["Country", "Age", "Gender", "Race", "Count"]
print(tabulate(results, headers=headers, tablefmt="grid"))


+-----------+-------+----------+----------+---------+
| Country   |   Age | Gender   | Race     |   Count |
+===========+=======+==========+==========+=========+
| India     |    35 | F        | Asian    |      58 |
+-----------+-------+----------+----------+---------+
| India     |    23 | M        | Hispanic |      56 |
+-----------+-------+----------+----------+---------+
| Canada    |    32 | F        | White    |      54 |
+-----------+-------+----------+----------+---------+
| Usa       |    68 | F        | Black    |      53 |
+-----------+-------+----------+----------+---------+
| Usa       |    80 | F        | White    |      53 |
+-----------+-------+----------+----------+---------+
| India     |    57 | F        | Asian    |      52 |
+-----------+-------+----------+----------+---------+
| Usa       |    38 | F        | Asian    |      52 |
+-----------+-------+----------+----------+---------+
| Usa       |    44 | F        | Other    |      51 |
+-----------+-------+-------

In [ ]:
# ✅ Query 20 – Complex
#  Top 5 Violations with Highest Arrest Rates
cursor.execute("""
WITH arrest_stats AS (
    SELECT violation,
           COUNT(*) AS total_stops,
           SUM(is_arrested) AS total_arrests,
           ROUND(SUM(is_arrested)/COUNT(*) * 100, 2) AS arrest_rate
    FROM policelog
    GROUP BY violation
    HAVING COUNT(*) > 10
),
ranked_arrests AS (
    SELECT *,
           RANK() OVER (ORDER BY arrest_rate DESC) AS rank_position
    FROM arrest_stats
)
SELECT violation, total_stops, total_arrests, arrest_rate
FROM ranked_arrests
WHERE rank_position <= 5;
""")
results = cursor.fetchall()
headers = ["Violation", "Total Stops", "Arrests", "Arrest Rate (%)"]
print(tabulate(results, headers=headers, tablefmt="grid"))


+-------------+---------------+-----------+-------------------+
| Violation   |   Total Stops |   Arrests |   Arrest Rate (%) |
+=============+===============+===========+===================+
| Dui         |         13075 |      6610 |             50.55 |
+-------------+---------------+-----------+-------------------+
| Seatbelt    |         13007 |      6573 |             50.53 |
+-------------+---------------+-----------+-------------------+
| Speeding    |         13150 |      6598 |             50.17 |
+-------------+---------------+-----------+-------------------+
| Signal      |         13112 |      6549 |             49.95 |
+-------------+---------------+-----------+-------------------+
| Other       |         13194 |      6516 |             49.39 |
+-------------+---------------+-----------+-------------------+
